# LeLM — Fine-tuned LLM for NBA Hot Takes

**Prerequisites**: Upload `train.jsonl` and `val.jsonl` to `Google Drive > MyDrive > LeLM/`

**Runtime**: GPU > T4 (free) for Qwen3-8B, or A100 (Pro) for Qwen3-14B

In [16]:
# Step 1: Install + Mount Drive + Config
!pip install -q unsloth transformers trl datasets peft bitsandbytes

from google.colab import drive
drive.mount('/content/drive')

import os, json, torch
from pathlib import Path

DRIVE_DIR = '/content/drive/MyDrive/LeLM'
TRAIN_FILE = Path(DRIVE_DIR) / 'train.jsonl'
VAL_FILE = Path(DRIVE_DIR) / 'val.jsonl'
OUTPUT_DIR = os.path.join(DRIVE_DIR, 'lelm-adapter')

# Verify data exists
assert TRAIN_FILE.exists(), f'Missing {TRAIN_FILE} — upload train.jsonl to Drive/LeLM/'
assert VAL_FILE.exists(), f'Missing {VAL_FILE} — upload val.jsonl to Drive/LeLM/'
print(f'Train: {sum(1 for _ in open(TRAIN_FILE))} examples')
print(f'Val:   {sum(1 for _ in open(VAL_FILE))} examples')

# Auto-detect GPU and pick model
gpu_name = torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'
vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
print(f'GPU: {gpu_name} ({vram_gb:.1f} GB)')

if 'A100' in gpu_name:
    MODEL_NAME = 'unsloth/Qwen3-14B-bnb-4bit'
elif 'T4' in gpu_name:
    MODEL_NAME = 'unsloth/Qwen3-8B-bnb-4bit'
else:
    MODEL_NAME = 'unsloth/Qwen3-4B-bnb-4bit'
print(f'Model: {MODEL_NAME}')

MAX_SEQ_LENGTH = 2048

SYSTEM_PROMPT = (
    'You are an unapologetically bold NBA analyst who lives for hot takes. '
    'You speak with absolute conviction, back up your claims with stats and game knowledge, '
    "but aren't afraid to be controversial. You have strong opinions on player legacies, "
    'team strategies, and playoff predictions. Your style is passionate, entertaining, '
    "and occasionally provocative — like a mix of Skip Bayless's confidence, Charles Barkley's "
    "humor, and Zach Lowe's basketball IQ. Never hedge. Never be boring. Every take should "
    'make someone want to argue with you.'
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Train: 2434 examples
Val:   129 examples
GPU: Tesla T4 (15.6 GB)
Model: unsloth/Qwen3-8B-bnb-4bit


In [17]:
# Step 2: Load model + LoRA
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,
    load_in_4bit=True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=64,
    lora_alpha=128,
    lora_dropout=0,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'],
    bias='none',
    use_gradient_checkpointing='unsloth',
)

==((====))==  Unsloth 2026.2.1: Fast Qwen3 patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.35. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
# Step 3: Train
from datasets import Dataset
from trl import SFTTrainer, SFTConfig

train_dataset = Dataset.from_list([json.loads(l) for l in open(TRAIN_FILE)])
val_dataset = Dataset.from_list([json.loads(l) for l in open(VAL_FILE)])
print(f'Train: {len(train_dataset)} | Val: {len(val_dataset)}')

def formatting_func(example):
    return [tokenizer.apply_chat_template(
        example['messages'], tokenize=False, add_generation_prompt=False
    )]

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    args=SFTConfig(
        output_dir=OUTPUT_DIR,
        num_train_epochs=3,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        lr_scheduler_type='cosine',
        warmup_steps=10,
        optim='adamw_8bit',
        weight_decay=0.01,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=10,
        save_strategy='epoch',
        eval_strategy='epoch',
        seed=42,
        max_seq_length=MAX_SEQ_LENGTH,
        dataset_text_field='text',
    ),
    formatting_func=formatting_func,
)

trainer.train()

# Save adapter to Drive
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f'Adapter saved to {OUTPUT_DIR}')

In [ ]:
# Step 4: Inference
FastLanguageModel.for_inference(model)

def generate(prompt, max_new_tokens=512):
    messages = [
        {'role': 'system', 'content': SYSTEM_PROMPT},
        {'role': 'user', 'content': prompt},
    ]
    inputs = tokenizer.apply_chat_template(
        messages, tokenize=True, add_generation_prompt=True, return_tensors='pt'
    ).to(model.device)
    outputs = model.generate(
        input_ids=inputs, max_new_tokens=max_new_tokens,
        temperature=0.8, top_p=0.9, do_sample=True,
    )
    return tokenizer.decode(outputs[0][inputs.shape[-1]:], skip_special_tokens=True).strip()

demos = [
    'Give me your hottest LeBron James take.',
    'Is Nikola Jokic the best player in the NBA right now?',
    "Who's the most overrated player in the league?",
    'Give me your boldest Finals prediction.',
    'Is the 3-point revolution ruining basketball?',
    'Give me your hottest Kevin Durant take.',
    'Is KD the best scorer in NBA history?',
]

for prompt in demos:
    print(f'\n>> {prompt}')
    print('-' * 40)
    print(generate(prompt))
    print()

In [ ]:
# Interactive — type your own prompts
prompt = 'Was KD the better player than Steph when he was on the Warriors?'  # @param {type:"string"}
print(generate(prompt))